In [1]:
# Importation des modules nécessaires

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
# Définition de l'espace de travail

import os
os.chdir(r"C:\Users\victo\Documents\DataScientest\Data Analyst\1000 - Travail de Groupe\Workspace\04 - Workspace - Machine Learning")

In [37]:
# Création du DataFrame

vgsales = pd.read_csv("vgsales_cleaned_franchise.csv")
display(vgsales.head())
print(len(vgsales))
print('\n')
print(vgsales.isna().sum())

,Rank,Name,Franchise,basename,Genre,Platform,Publisher,Developer,Year,Estimated_Sales
0,1,Wii Sports,Undefined Franchise,wii-sports,Sports,Wii,Nintendo,Nintendo EAD,2006.0,82.86
1,2,Super Mario Bros.,Mario,super-mario-bros,Platform,NES,Nintendo,Nintendo EAD,1985.0,40.24
2,3,Mario Kart Wii,Mario,mario-kart-wii,Racing,Wii,Nintendo,Nintendo EAD,2008.0,37.14
3,4,PlayerUnknown's Battlegrounds,Undefined Franchise,playerunknowns-battlegrounds,Shooter,PC,PUBG Corporation,PUBG Corporation,2017.0,36.60
4,5,Wii Sports Resort,Undefined Franchise,wii-sports-resort,Sports,Wii,Nintendo,Nintendo EAD,2009.0,33.09


21233


Rank                 0
Name                 0
Franchise            0
basename             0
Genre                0
Platform             0
Publisher          581
Developer          651
Year                29
Estimated_Sales      0
dtype: int64


In [38]:
# Suppressions de valeur manquantes dans la colonne Year
# Nous encoderons cette colonne avec un StandardScaler, qui conserve les valeur manquantes
# Notre algorithme de ML ne pourra pas les gérer

vgsales.dropna(subset = ['Year'], inplace = True)
vgsales.isna().sum()

Rank                 0
Name                 0
Franchise            0
basename             0
Genre                0
Platform             0
Publisher          562
Developer          650
Year                 0
Estimated_Sales      0
dtype: int64

In [39]:
# Suppression de la colonne Name, qui a une cardinalité trop importante pour l'encodage
# Suppression de la colonne Rank, qui donne trop d'informations sur les données
# Suppression de la colonne basename, pour le même motif que la colonne Name

vgsales = vgsales.drop('Name', axis = 1)
vgsales = vgsales.drop('Rank', axis = 1)
vgsales = vgsales.drop('basename', axis = 1)
vgsales.head()


,Franchise,Genre,Platform,Publisher,Developer,Year,Estimated_Sales
0,Undefined Franchise,Sports,Wii,Nintendo,Nintendo EAD,2006.0,82.86
1,Mario,Platform,NES,Nintendo,Nintendo EAD,1985.0,40.24
2,Mario,Racing,Wii,Nintendo,Nintendo EAD,2008.0,37.14
3,Undefined Franchise,Shooter,PC,PUBG Corporation,PUBG Corporation,2017.0,36.60
4,Undefined Franchise,Sports,Wii,Nintendo,Nintendo EAD,2009.0,33.09


In [40]:
# Séparation des varirables explicatives et de la variable cible dans deux DataFrame différents

feats = vgsales.drop('Estimated_Sales', axis = 1)
target = vgsales['Estimated_Sales']

display(feats)
display(target)

,Franchise,Genre,Platform,Publisher,Developer,Year
0,Undefined Franchise,Sports,Wii,Nintendo,Nintendo EAD,2006.0
1,Mario,Platform,NES,Nintendo,Nintendo EAD,1985.0
2,Mario,Racing,Wii,Nintendo,Nintendo EAD,2008.0
3,Undefined Franchise,Shooter,PC,PUBG Corporation,PUBG Corporation,2017.0
4,Undefined Franchise,Sports,Wii,Nintendo,Nintendo EAD,2009.0
...,...,...,...,...,...,...
21228,Undefined Franchise,Puzzle,DS,505 Games,Crush Digital,2010.0
21229,Undefined Franchise,Strategy,PC,ValuSoft,ValuSoft,2007.0
21230,Tales,Action,PSP,Sony Computer Entertainment,Acquire,2007.0
21231,Undefined Franchise,Shooter,PC,Atari,Kamehan Studios,2002.0


0        82.86
1        40.24
2        37.14
3        36.60
4        33.09
         ...  
21228     0.00
21229     0.00
21230     0.00
21231     0.00
21232     0.00
Name: Estimated_Sales, Length: 21204, dtype: float64

In [41]:
# Séparation du des données en jeu de test et jeu d'entraînement

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2, random_state = 42)

print(X_train.shape)
print(X_test.shape)

(16963, 6)
(4241, 6)


In [42]:
# Encoding
# Franchise, Genre, Platform, Publisher, Developer : OneHotEncoder

X_train_ohe = X_train[['Franchise', 'Genre', 'Platform', 'Publisher', 'Developer']]
X_test_ohe = X_test[['Franchise', 'Genre', 'Platform', 'Publisher', 'Developer']]

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop = None, handle_unknown = 'ignore')

X_train_ohe = pd.DataFrame(ohe.fit_transform(X_train_ohe).toarray(), columns = ohe.get_feature_names_out())
X_test_ohe = pd.DataFrame(ohe.transform(X_test_ohe).toarray(), columns = ohe.get_feature_names_out())

print(X_train_ohe.shape)
print(X_test_ohe.shape)

(16963, 4116)
(4241, 4116)


In [43]:
# Encoding
# Year : Standard Scaler

from sklearn.preprocessing import StandardScaler

X_train_sc = np.asarray(X_train['Year']).reshape(-1,1)
X_test_sc = np.asarray(X_test['Year']).reshape(-1, 1)

sc = StandardScaler()
X_train_sc = pd.DataFrame(sc.fit_transform(X_train_sc), columns = ['Year'])
X_test_sc = pd.DataFrame(sc.transform(X_test_sc), columns = ['Year'])

print(X_train_sc.shape)
print(X_test_sc.shape)

(16963, 1)
(4241, 1)


In [44]:
# Rassemblement des DataFrames encodés

X_train = pd.concat([X_train_ohe, X_train_sc], ignore_index = False, axis = 1)
X_test = pd.concat([X_test_ohe, X_test_sc], ignore_index = False, axis = 1)

print(X_train.shape)
print(X_test.shape)

(16963, 4117)
(4241, 4117)


In [45]:
print(X_train.isna().sum().sum())
print(X_test.isna().sum().sum())

0
0


In [11]:
# RandomForestResgressor

from sklearn.ensemble import RandomForestRegressor

rfreg = RandomForestRegressor(random_state = 42, max_depth = 2)
rfreg.fit(X_train, y_train)

print("Score du modèle de Random Forest Classifier sur le jeu de données d'entraînement:", rfreg.score(X_train, y_train))
print("Score du modèle de Random Forest Classifier sur le jeu de données de test:", rfreg.score(X_test, y_test))

Score du modèle de Random Forest Classifier sur le jeu de données d'entraînement: 0.26999698156035934
Score du modèle de Random Forest Classifier sur le jeu de données de test: 0.157552232347817


In [ ]:
# RandomForestResgressor

from sklearn.ensemble import RandomForestRegressor

rfreg = RandomForestRegressor(random_state = 42)
rfreg.fit(X_train, y_train)

print("Score du modèle de Random Forest Classifier sur le jeu de données d'entraînement:", rfreg.score(X_train, y_train))
print("Score du modèle de Random Forest Classifier sur le jeu de données de test:", rfreg.score(X_test, y_test))

# Le modèle est sur-entraîné mais performe mieux que la régression linéaire
# Attention avec la méthode d'encodage, on a un gros risque de fuite de données

In [46]:
# RandomForestResgressor

from sklearn.ensemble import RandomForestRegressor

rfreg = RandomForestRegressor(random_state = 42, max_depth = 4)
rfreg.fit(X_train, y_train)

print("Score du modèle de Random Forest Classifier sur le jeu de données d'entraînement:", rfreg.score(X_train, y_train))
print("Score du modèle de Random Forest Classifier sur le jeu de données de test:", rfreg.score(X_test, y_test))

# Le modèle est sur-entraîné mais performe mieux que la régression linéaire
# Attention avec la méthode d'encodage, on a un gros risque de fuite de données

Score du modèle de Random Forest Classifier sur le jeu de données d'entraînement: 0.4630329449937787
Score du modèle de Random Forest Classifier sur le jeu de données de test: 0.20016333136021924


In [52]:
# Interprétation: Mesure de l'importance de chaque variable

feature_importance = pd.DataFrame({'Variable':X_train.columns, 'Importance':rfreg.feature_importances_}).sort_values('Importance', ascending=False)

print('Top des variables par capacité prédictive')
display(feature_importance[feature_importance['Importance'] > 0.02])

Top des variables par capacité prédictive


,Variable,Importance
2906,Developer_Nintendo EAD,0.197790
4116,Year,0.152694
123,Genre_Sports,0.145480
101,Franchise_Undefined Franchise,0.123468
2106,Developer_Game Freak,0.088829
157,Platform_Wii,0.046915
1514,Developer_Bullet Proof Software,0.031989
2791,Developer_Mojang AB,0.030278
2174,Developer_Good Science Studio,0.025620
